# 系统默认编码

In [ ]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding("utf-8")
sys.stdout = stdout

# 导入库

In [ ]:
import pandas as pd
import numpy as np

# 读取数据

## 获取表头

In [ ]:
# 读取第一行表头
f = open("./data.csv", "r")
line = f.readline()
f.close()

In [ ]:
# 获取列名
headerList = line.replace('","', ',').split(",")

# 首尾两个列名需要处理下
headerList[0] = "id"
headerList[-1] = headerList[-1].strip()[0:-1]

In [ ]:
# 打印所有的列名
print headerList

## pandas读取csv文件，并转换为DataFrame

HBuilder打开csv文件，发现编码方式为<strong>GB18030</strong>

In [ ]:
dataSet = pd.read_csv("./data.csv", delimiter = ",", encoding = "GB18030", skiprows=1, names = headerList)

## 数据预览

### 打印首尾数行

In [ ]:
# 前5行
dataSet.head(5)

In [ ]:
# 后5行
dataSet.tail(5)

### 获取数据规模

In [ ]:
print u"数据规模为：", dataSet.shape[0]
print u"特征个数为：", dataSet.shape[1]

### 查看数据信息

一共有83个数值型特征，和7个Python对象型特征(其实就是字符串)
float64(70), int64(13), object(7)

In [ ]:
dataSet.info()

# 数据清洗

## 删除无关特征

观察非数值型特征
<p><li>reg_preference_for_trad：城市等级，取值范围有：一线、二线、三线、境外等。应该是有价值的参考信息<p></li>
<p><li>source：均为xs，无关特征<p></li>
<p><li>id_name：人名，无关特征<p></li>
<p><li>latest_query_time：日期参数，因为不涉及时间序列分析，所以应该为无关特征<p></li>
<p><li>loans_latest_time：日期信息，因为不涉及时间序列分析，所以应该为无关特征<p></li>

带有id、no字样的特征，也为无关特征(id是自己添加上去的，原始数据没有这个字段)
<p><li>id<p></li>
<p><li>custid<p></li>
<p><li>trade_no<p></li>
<p><li>bank_card_no<p></li>

In [ ]:
# 删除无关特征
dropCols = ["source", "id_name", "latest_query_time", "loans_latest_time", "id", "custid", "trade_no", "bank_card_no"]
dataSet.drop(columns = dropCols, inplace = True)
print 

## 过滤失效特征

可以看到，"student_feature"有近63%的数据为缺失值，可以断定为是一个失效特征

In [ ]:
# 属性字典，键为特征名，值为(缺失值，缺失值所占比例cd
featureInfo = {}

# 遍历所有的特征
for i in range(dataSet.shape[1]):
    
    series = dataSet.iloc[:, i]
    
    nullCounter = 0.0
    
    # 遍历所有的样本点
    for j in range(dataSet.shape[0]):
        
        # np.isnan()的形参只能接受数值型
        try:  
            if np.isnan(series[j]): 
                nullCounter += 1
        
        # 非空字符
        except:
            if series[j] != "NA":
                pass
    
    # 本特征的取值统计完毕
    featureInfo[dataSet.columns.tolist()[i]] = (nullCounter, nullCounter / dataSet.shape[0] * 100)

# 根据缺失值所占比例逆序排序
sorted(featureInfo.items(), key = lambda item: item[1][1], reverse = True)

In [ ]:
# 删除student_feature
dataSet.drop(columns = "student_feature", inplace = True)

## 剔除失效样本

这里定义样本的缺失值在10个以上时，可以认为是一个失效样本，应该予以删除。

In [ ]:
dataSet.dropna(thresh = 70, inplace = True)
print u"数据规模为：", dataSet.shape[0]
print u"特征个数为：", dataSet.shape[1]

## 缺失值填充

填充缺失值之前，对数据再进行一次预览

对于数值型数据的缺失值，传入平均数；对于非数值型(object类型)reg_preference_for_trad特征，需要转换成独热编码
<ol>
<li>在过滤失效特征的步骤中，发现仅仅只出现过两次缺失值，对于这类特征的缺失值，可以直接删除
<li>在剔除失效样本之后，发现reg_preference_for_trady特征的描述为“reg_preference_for_trad  4453  non-null object”，已经没有缺失值
故，不需要对reg_preference_for_trady特征进行缺失值的填充处理
<li>status标签的描述为：“status  4453 non-null int64”，即标签中也没有缺失值。
</ol>

In [ ]:
len(dataSet.loc[:, "reg_preference_for_trad"])
len(dataSet.loc[:, "reg_preference_for_trad"].tolist())

In [ ]:
# 提取 reg_preference_for_trad
reg_preList = dataSet.loc[:, "reg_preference_for_trad"]
dataSet.drop(columns = "reg_preference_for_trad", inplace = True)

In [ ]:
# 填充缺失值
dataSet.fillna(dataSet.mean(), inplace = True)

In [ ]:
from sklearn import preprocessing

# 标签二值化
dummData = preprocessing.LabelBinarizer().fit_transform(reg_preList) 
dummDataFrame = pd.DataFrame(dummData, columns=["一线城市","三线城市","二线城市","其它城市","境外"])

# 合并数据集，得到最终数据
dataSet.reset_index(drop=True, inplace=True) 
dummDataFrame.reset_index(drop=True, inplace=True) 
dataSet = pd.concat([dataSet, dummDataFrame], axis=1)

# 特征提取(待续)